In [1]:
# %pip install pandas
# %pip install xlrd
import pandas as pd
import re

In [2]:
# o excel possui comentários até a linha 41 e a linha 43 está vazia
df = pd.read_excel("cmed_final_v3.xlsx")
df.head(1)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,rec,13,Por subs (1 a 4),Por tipo(1 a 10),Probabilidade
0,22,359,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,1023504310059,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,150,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,Tarja Vermelha (**),1,1,2.5
1,23,76,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501115110025705,1063902720108,NESINA MET,"(12,5 + 1000,0) MG COM REV CT BL AL AL X 60",60,A10N3 - ASSOCIAÇÕES DE INIBIDORES DPP-IV COM B...,Novo,228.00,303.77,Liberado,Tarja Vermelha,1,1,2.5
2,24,470,CARBAMAZEPINA,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,526515003110311,1006800850070,TEGRETOL,200 MG COM CT BL AL PLAS PVC TRANS X 60,60,N3A - ANTIEPILÉPTICOS,Novo,64.54,89.22,C1,Tarja Vermelha sob restrição,1,1,2.5
3,25,496,CLORIDRATO DE SERTRALINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528519120163307,1256802880219,TRELIM,100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60,60,N6A4 - ANTIDEPRESSIVOS SSRI,Similar,290.24,401.24,C1,Tarja Vermelha sob restrição,1,1,2.5
4,26,300,EZETIMIBA,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,543817070024603,1351700300049,ZETERINA,10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60,60,C10A9 - TODOS OUTROS REGULADORES DE COLESTEROL...,Similar,200.62,277.35,Liberado,Tarja Vermelha,1,1,2.5


In [3]:
# selecionar somente linhas desejadas
df = df.filter(items=[1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 'rec', 'Probabilidade'])

# renomear algumas colunas
df = df.rename(columns={1: 'SUBSTÂNCIA', 2: 'CNPJ', 3: 'LABORATÓRIO', 4: 'GGREM', 6: 'PRODUTO', 7: 'APRESENTAÇÃO', 9: 'CLASSE TERAPÊUTICA', 10: 'TIPO', 11: 'CUSTO', 12: 'PREÇO', 'rec': 'CONTROLE', 'Probabilidade': 'PROBABILIDADE'})
df.head()

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,GGREM,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO,CUSTO,PREÇO,CONTROLE,PROBABILIDADE
0,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,2.5
1,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501115110025705,NESINA MET,"(12,5 + 1000,0) MG COM REV CT BL AL AL X 60",A10N3 - ASSOCIAÇÕES DE INIBIDORES DPP-IV COM B...,Novo,228.00,303.77,Liberado,2.5
2,CARBAMAZEPINA,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,526515003110311,TEGRETOL,200 MG COM CT BL AL PLAS PVC TRANS X 60,N3A - ANTIEPILÉPTICOS,Novo,64.54,89.22,C1,2.5
3,CLORIDRATO DE SERTRALINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528519120163307,TRELIM,100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60,N6A4 - ANTIDEPRESSIVOS SSRI,Similar,290.24,401.24,C1,2.5
4,EZETIMIBA,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,543817070024603,ZETERINA,10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60,C10A9 - TODOS OUTROS REGULADORES DE COLESTEROL...,Similar,200.62,277.35,Liberado,2.5


In [7]:
# # df.iloc[1:100, 4]

# pt1 = r'SUS|COM|CAP|SOL|XPE|CREM|POM|INJ|IM|GEL|EMU|PAST|LOCAO|XAMP|DRG|$'

# cont = {'SUS': 0, 'COM': 0, 'CAP': 0, 'SOL': 0, 'XPE': 0, 'CREM': 0, 'POM': 0}
# for a in df['apresentacao']:
#   # if re.search(pt1, a) and re.search(pt1, a).group() == 'COM':
#   #   print(re.split(pt1, a)[0], '=>', re.search(pt1, a).group(), '|', a)
#   print(re.split(pt1, a)[0], '=>', re.search(pt1, a).group(), '|', a)

#   # if re.search(pt1, a) and re.search(pt1, a).group() in cont:
#   #   cont[re.search(pt1, a).group()] += 1
#   # else:
#   #   print(re.split(pt1, a)[0], '=>', re.search(pt1, a).group(), '|', a)

# print(cont)

# pt1 = r'SUS|COM|CAP|SOL|XPE|CREM|POM|INJ|IM|GEL|EMU|PAST|LOCAO|XAMP|DRG|$'

# for i in range(df.shape[0]):
#   apres = df['apresentacao'][i]
#   rg = re.search(pt1, apres).group()
#   quant = df['quantidade'][i]
#   print(quant, rg)

# [e for e in df['apresentacao']]
# pt = 'MG'
# re.search(pt, df['apresentacao'][41]).group()

# for a in df['apresentacao']:
#   a = a.replace(' ', '')
  
#   m2 = re.split('X\s', a)
#   q = m2[-1] if m2 else ''

#   print(a, '=> QUANTIDADE:', q)


# [e for e in df['CONTROLE']]
df.loc[0, 'APRESENTAÇÃO']

'50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML\xa0'

Criando colunas "DOSAGEM", "APRESENTACAO", "CONTROLE", "QUANTIDADE", "CLASSE"

In [8]:
ps = ['MG\s\/\sML', 'MEQ/L', 'ML/ML', 'MG/ML', 'MG/G', 'GV/ML', 'G/L', 'G/G', 'MG\)', 'MG', 'MCG', 'PCC', 'G/ML', 'TRU', 
      'MU/ML', 'UI/ML', 'SOL\sINJ', 'UI/G', 'U/G', 'U/U', '\sUI\s', '\sML\s', '\sU\s', '\sG\s', '\s1G\s', '1G\s', '$']
pt3 = r'POM|CREM|GEL|MOLE|EMU|PAST|LOCAO|XAMP'
pt4 = r'SOL|XPE|SUS|INJ|ML|IM'
pt5 = r' - '

dosagem = []
apresentacao = []
# controle = []
classe = []
quantidade = []


# for a in df['APRESENTAÇÃO']:
for i in range(df.shape[0]):
  a = df.loc[i, 'APRESENTAÇÃO'].replace(' ', '')
  
  # DOSAGEM
  d = ''
  for p in ps:
    m1 = re.search(p, a)
    if m1:
      d = re.split(p, a)[0] + m1.group()
      break
  d = d.strip()
  
  # QUANTIDADE
  m2 = re.split('X\s', a)
  q = m2[-1] if m2 else ''
  q = q.strip()

  # APRESENTACAO
  m3 = re.search(pt3, a)
  m4 = re.search(pt4, a)
  if m3: t = 'Semissólido'
  elif m4: t = 'Líquido'
  else: t = 'Sólido'

  dosagem.append(d)
  quantidade.append(q)
  apresentacao.append(t)
  # print(a, '=> DOSAGEM:', d, '| QUANTIDADE:', q, '| APRESENTACAO:', t)

  a = df.loc[i, 'CLASSE TERAPÊUTICA'].replace(' ', '')
  # co = re.split(pt5, a)[0][:2]
  # co = co if co in ['A2', 'A3', 'B1', 'B2', 'C1', 'C2', 'AM'] else 'Não controlado'
  # controle.append(co)
  
  cl = re.split(pt5, a)[1]
  classe.append(cl)
  # print(a, '=> CONTROLE:', co, '| CLASSE:', cl)
  
df['DOSAGEM'] = dosagem
df['APRESENTACAO'] = apresentacao
# df['CONTROLE'] = controle # rec
df['QUANTIDADE'] = quantidade
df['CLASSE'] = classe

In [9]:
df.head()

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,GGREM,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO,CUSTO,PREÇO,CONTROLE,PROBABILIDADE,DOSAGEM,APRESENTACAO,QUANTIDADE,CLASSE
0,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,2.5,50 MG/ML,Líquido,150 ML,PENICILINAS ORAIS DE AMPLO ESPECTRO
1,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501115110025705,NESINA MET,"(12,5 + 1000,0) MG COM REV CT BL AL AL X 60",A10N3 - ASSOCIAÇÕES DE INIBIDORES DPP-IV COM B...,Novo,228.00,303.77,Liberado,2.5,"(12,5 + 1000,0) MG",Sólido,60,ASSOCIAÇÕES DE INIBIDORES DPP-IV COM BIGUANIDAS
2,CARBAMAZEPINA,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,526515003110311,TEGRETOL,200 MG COM CT BL AL PLAS PVC TRANS X 60,N3A - ANTIEPILÉPTICOS,Novo,64.54,89.22,C1,2.5,200 MG,Sólido,60,ANTIEPILÉPTICOS
3,CLORIDRATO DE SERTRALINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528519120163307,TRELIM,100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60,N6A4 - ANTIDEPRESSIVOS SSRI,Similar,290.24,401.24,C1,2.5,100 MG,Sólido,60,ANTIDEPRESSIVOS SSRI
4,EZETIMIBA,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,543817070024603,ZETERINA,10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60,C10A9 - TODOS OUTROS REGULADORES DE COLESTEROL...,Similar,200.62,277.35,Liberado,2.5,10 MG,Sólido,60,TODOS OUTROS REGULADORES DE COLESTEROL E TRIGL...


Filtro dos medicamentos

In [ ]:
# df_filtro = df.loc[
#   (df.PREÇO < 300) &
#   (df.PRODUTO.isin(['BUSCOPAN', 'DEPOSTERON', 'ACCUVIT', 'ACEBROFILINA']))
# ]
# df_filtro

Uma substância por laboratório ou somente uma substância

In [ ]:
# df_filtro.groupby('CNPJ').first().groupby('SUBSTÂNCIA').first()
# df_filtro.groupby('CNPJ').first()

'N6A4 - ANTIDEPRESSIVOS SSRI'

Laboratórios

,CNPJ,LABORATÓRIO,NOME
0,57.507.378/0003-65,EMS S/A,EMS
1,60.397.775/0001-74,TAKEDA PHARMA LTDA.,TAKEDA
2,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,NOVARTIS
3,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,PRATI
4,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,ALTHAIA
5,46.070.868/0036-99,LABORATÓRIOS PFIZER LTDA,PFIZER
6,60.659.463/0029-92,ACHÉ LABORATÓRIOS FARMACÊUTICOS S.A,ACHÉ
7,60.318.797/0001-00,ASTRAZENECA DO BRASIL LTDA,ASTRAZENECA
8,60.831.658/0001-77,BOEHRINGER INGELHEIM DO BRASIL QUÍMICA E FARMA...,BOEHRINGER
9,61.190.096/0001-92,EUROFARMA LABORATÓRIOS S.A.,EUROFARMA


In [11]:
import psycopg2

def exec(sql):
  con = None
  cur = None
  try:
    con = psycopg2.connect(database = "drogasystem", user = "oscar", password = "oscar", host = "127.0.0.1", port = "5432")
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    cur.close()
    con.close()
  except (Exception, psycopg2.Error) as error:
    if cur == None: cur.rollback()
    print("Error:", error)
  finally:
    if cur == None: cur.close()
    if con == None: con.close()


In [26]:
# nomes dos laboratórios
df_lab = df.filter(items=['CNPJ', 'LABORATÓRIO']).drop_duplicates(subset=['CNPJ']).reset_index(drop=True)

nome = []
for e in df_lab['LABORATÓRIO']:
  s = e
  s = s.replace('LABORATÓRIOS', '').strip()
  s = s.replace('LABORATÓRIO', '').strip()
  s = s.replace('LABORATORIO', '').strip()
  s = s.replace('FARMACÊUTICO', '').strip()
  s = s.replace('FARMACÊUTICA', '').strip()
  s = s.replace('INDUSTRIAL', '').strip()
  s = s.replace('INDÚSTRIA', '').strip()
  s = s.replace('INDUSTRIA', '').strip()
  s = s.replace('PRODUTOS', '').strip()
  s = s.replace('PRODUTOS', '').strip()
  s = s[:3] + re.split(r'\.|\s', s[3:])[0]
  s = s.replace('  ', ' ').strip()
  # print(s, '=>', e)
  nome.append(s)
df_lab['NOME'] = nome
df_lab.head(10)

# insert laboratório
sql = f"insert into laboratorio values \n  (default, '{df_lab.iloc[0, 2]}', '{df_lab.iloc[0, 1]}', '{df_lab.iloc[0, 0]}')"
for i in range(1, df_lab.shape[0]):
  sql += f"\n, (default, '{df_lab.iloc[i, 2]}', '{df_lab.iloc[i, 1]}', '{df_lab.iloc[i, 0]}')"
sql += ";"

# print(sql)
exec(sql)

,id,nome,razao,cnpj
0,1,EMS,EMS S/A,57.507.378/0003-65
1,2,TAKEDA,TAKEDA PHARMA LTDA.,60.397.775/0001-74
2,3,NOVARTIS,NOVARTIS BIOCIENCIAS S.A,56.994.502/0001-30
3,4,PRATI,PRATI DONADUZZI & CIA LTDA,73.856.593/0001-66
4,5,ALTHAIA,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,48.344.725/0007-19


In [27]:
# select laboratório
con = psycopg2.connect(database = "drogasystem", user = "oscar", password = "oscar", host = "127.0.0.1", port = "5432")
cur = con.cursor()
cur.execute("select * from laboratorio order by 1;")
df_lab = pd.DataFrame(cur.fetchall(), columns=['id', 'nome', 'razao', 'cnpj'])
df_lab.head()

# insert medicamento
laboratorio_id = df_lab[df_lab['cnpj'] == df.iloc[0, 1]]['id'].reset_index(drop=True)[0]
sql  = f"insert into medicamento values \n"
sql += f"  (default, '{laboratorio_id}', '{df.iloc[0, 4]}', '{df.iloc[0, 0]}', '{df.iloc[0, 12]}', '{df.iloc[0, 13]}', '{df.iloc[0, 7]}'"
sql += f", '{df.iloc[0, 10]}', '{df.iloc[0, 14]}', '{df.iloc[0, 3]}', '{df.iloc[0, 15]}', '{df.iloc[0, 8]}', '{df.iloc[0, 9]}')"
for i in range(1, df.shape[0]):
  laboratorio_id = df_lab[df_lab['cnpj'] == df.iloc[i, 1]]['id'].reset_index(drop=True)[0]
  sql += f"\n, (default, '{laboratorio_id}', '{df.iloc[i, 4]}', '{df.iloc[i, 0]}', '{df.iloc[i, 12]}', '{df.iloc[i, 13]}', '{df.iloc[i, 7]}'"
  sql += f", '{df.iloc[i, 10]}', '{df.iloc[i, 14]}', '{df.iloc[i, 3]}', '{df.iloc[i, 15]}', '{df.iloc[i, 8]}', '{df.iloc[i, 9]}')"
sql += ";"

# print(sql)
exec(sql)

# df_lab[df_lab['cnpj'] == '57.507.378/0003-65']['id']
# df_lab[df_lab['cnpj'] == df.iloc[0, 1]]['id'].reset_index(drop=True)[0]

In [28]:
cur.close()
con.close()